In [1]:
#This non-descript folder contains all the standalone python modules
from python import *

import pickle
import os
import numpy as np
import json
from ipyparallel import Client
from itertools import repeat

from sklearn.linear_model import LinearRegression

In [2]:
#Create data classes
#Year, Season
seasonsData = None
#RaceId, List of tuples of (driverId, constructorId, time)
qualiResultsData = None
#DriverId, name
driversData = None
#ConstructorId, name
constructorsData = None
#EngineId, name
enginesData = None

with open('data/seasonsData.txt', 'rb') as handle:
    seasonsData = pickle.load(handle)
    #print(seasonsData)
    
with open('data/qualiResultsData.txt', 'rb') as handle:
    qualiResultsData = pickle.load(handle)
    #print(qualiResultsData)
    
with open('data/driversData.txt', 'rb') as handle:
    driversData = pickle.load(handle)
    #print(driversData)
    
with open('data/constructorsData.txt', 'rb') as handle:
    constructorsData = pickle.load(handle)
    #print(constructorsData)
    
with open('data/enginesData.txt', 'rb') as handle:
    enginesData = pickle.load(handle)
    #print(enginesData)

In [3]:
entries = []
results = []
cleaner = F1DataCleaner(seasonsData, qualiResultsData, driversData, constructorsData, enginesData)
cleaner.constructDataset(entries, results)


In [4]:
#print(entries)

In [5]:
#print(results)

In [6]:
X = np.array(entries)
y = results
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.5665622944845082

In [7]:
reg.coef_

array([ 0.59852202,  0.65566164,  0.24871502, -0.03657039,  0.03692107,
        0.02323797])

In [8]:
newDrivers = json.load(open('data/newDrivers.json'))["drivers"]
newDrivers = {int(did): cid for did, cid in newDrivers.items()}

#print(newDrivers)
driversToWrite = {}
for did, cid in newDrivers.items():
    driversToWrite[int(did)] = {}
    driversToWrite[int(did)]["name"] = cleaner.drivers[int(did)].name
    driversToWrite[int(did)]["constructor"] = cleaner.drivers[int(did)].constructor.name
with open('../F1PredictWeb/driversDescribe.json', 'w') as fp:
    json.dump(driversToWrite, fp)

In [9]:
circuit = 32
circuitName = "Mexican"

with open('../F1PredictWeb/circuitName.txt', 'w') as fp:
    fp.write(circuitName);

In [10]:
predictedEntrants = []

for did, cid in newDrivers.items():
    if circuit not in cleaner.drivers[did].trackpwr:
        cleaner.drivers[did].trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.trackpwr:
        cleaner.drivers[did].constructor.trackpwr[circuit] = 0 #TODO maybe change defaults
    if circuit not in cleaner.drivers[did].constructor.engine.trackpwr:
        cleaner.drivers[did].constructor.engine.trackpwr[circuit] = 0 #TODO maybe change defaults
    
    entry = [
        cleaner.drivers[did].pwr,
        cleaner.drivers[did].constructor.pwr, 
        cleaner.drivers[did].constructor.engine.pwr,
        cleaner.drivers[did].trackpwr[circuit],
        cleaner.drivers[did].constructor.trackpwr[circuit],
        cleaner.drivers[did].constructor.engine.trackpwr[circuit]
    ]
    predictedEntrants.append(entry)
    

In [11]:
predictedResults = reg.predict(np.array(predictedEntrants))
print(predictedResults)

[-1.3462252   0.64128    -1.03111387 -0.95973793 -0.13685707  0.12061158
 -0.08399701 -0.05074259 -1.26909349  0.03778865  0.51061142 -0.18614802
  0.08502952  0.88276589 -0.13501726  0.67693847  0.18273322  0.39047527
  0.6526653   0.10827285]


In [12]:
for index, (did, cid) in enumerate(newDrivers.items()):
    print("{0} ({1}): {2}".format(cleaner.drivers[int(did)].name, cleaner.drivers[int(did)].constructor.name, predictedResults[index]))

Lewis Hamilton (Mercedes): -1.3462251953946944
Fernando Alonso (McLaren): 0.6412799952750861
Kimi Räikkönen (Ferrari): -1.0311138673906344
Sebastian Vettel (Ferrari): -0.9597379273331874
Romain Grosjean (Haas F1 Team): -0.13685706902572803
Nico Hülkenberg (Renault): 0.12061158489124733
Sergio Pérez (Force India): -0.08399701480664679
Daniel Ricciardo (Red Bull): -0.050742592402929075
Valtteri Bottas (Mercedes): -1.269093494434733
Kevin Magnussen (Haas F1 Team): 0.037788645296608026
Marcus Ericsson (Sauber): 0.5106114203283715
Max Verstappen (Red Bull): -0.18614802155049587
Carlos Sainz (Renault): 0.08502952324226064
Stoffel Vandoorne (McLaren): 0.8827658853015418
Esteban Ocon (Force India): -0.13501725757467672
Lance Stroll (Williams): 0.676938465665694
Pierre Gasly (Toro Rosso): 0.18273321694385414
Brendon Hartley (Toro Rosso): 0.3904752664749239
Sergey Sirotkin (Williams): 0.6526652963784737
Charles Leclerc (Sauber): 0.10827285236984464
